In [1]:
from pyrpipe import sra,mapping,assembly,qc,tools
#First get the srr accessions of the runs. For this one can use the python package pysradb or R package sradb
#runs=['SRR3098746','SRR3098745','SRR3098744'] #from the study SRP068369
runs=['SRR765545'] #small test
#set up directories
maize_data="/home/usingh/work/urmi/hoap/test/maize/maize_data"
workingDir="/home/usingh/work/urmi/hoap/test/maize"



Logs will be saved to 2020-01-16-16_48_04_pyrpipe.log


## Download data, pre-process

In [2]:
sraObjects=[]
for x in runs:
    thisSraOb=sra.SRA(x,workingDir)
    if thisSraOb.download_sra():
        sraObjects.append(thisSraOb)
    else:
        print("Download failed:"+x)
        
#perform fastq dump and qc

#create a Trimgalore object
tgOpts={"--cores": "10"}
tg=qc.Trimgalore(**tgOpts)

for x in sraObjects:
    #to fastq
    x.run_fasterqdump(delete_sra=True,**{"-e":"20","-f":"","-t":workingDir}) #use 20 threads
    #perform qc using trim galore
    x.perform_qc(tg)
    

        

$ prefetch -O /home/usingh/work/urmi/hoap/test/maize/SRR765545 SRR765545
Time taken:0:00:14
Downloaded file: /home/usingh/work/urmi/hoap/test/maize/SRR765545/SRR765545.sra 500.4 MB 
$ fasterq-dump -e 20 -f -t /home/usingh/work/urmi/hoap/test/maize -O /home/usingh/work/urmi/hoap/test/maize/SRR765545 -o SRR765545.fastq /home/usingh/work/urmi/hoap/test/maize/SRR765545/SRR765545.sra
Time taken:0:00:27
Performing QC using trim_galore
$ trim_galore --cores 10 --paired -o /home/usingh/work/urmi/hoap/test/maize/SRR765545 /home/usingh/work/urmi/hoap/test/maize/SRR765545/SRR765545_1.fastq /home/usingh/work/urmi/hoap/test/maize/SRR765545/SRR765545_2.fastq
Time taken:0:01:09


## Map using STAR

In [ ]:
starParams={"--outFilterType":"BySJout",
            "--runThreadN":"8",
            "--outSAMtype": "BAM SortedByCoordinate"
            }

star=mapping.Star(star_index="",**starParams) #provided index is invalid

#create star index
indexOut=maize_data+"/starindex"
inFasta=maize_data+"/Zea_mays.B73_RefGen_v4.dna.toplevel.1_10.fa"
star.build_index(indexOut,inFasta)



No STAR index provided. Please build index now to generate an index using build_index()....
$ STAR --runMode genomeGenerate --genomeDir /home/usingh/work/urmi/hoap/test/maize/maize_data/starindex --genomeFastaFiles /home/usingh/work/urmi/hoap/test/maize/maize_data/Zea_mays.B73_RefGen_v4.dna.toplevel.1_10.fa


## Transcript assembly using StringTie

In [ ]:
#Create object for stringtie. This will be used for all the bam files.
st=assembly.Stringtie(reference_gtf="maize_data/Zea_mays.B73_RefGen_v4.46.gtf")
gtfList=[]
for x in sraObjects:
    star_out_dir=star.perform_alignment(x,objectid=x.srr_accession)
    bam=star_out_dir+"/Aligned.sortedByCoord.out.bam"
    gtfList.append(st.perform_assembly(bam,objectid=x.srr_accession,**{"-p":"25"}))   

print(gtfList)


## lncRNA prediction using PLncPRO
We will use [PLncPRO](https://github.com/urmi-21/PLncPRO) for prediction of lncRNAs. Currently, PLncPRO is not integrated into `pyrpipe` so we will use the `pyrpipe_engine` module directly to execute.

In [ ]:
#import pyrpipe modules
from pyrpipe import pyrpipe_engine as pe
#install plncpro
pe.execute_command("pip install plncpro".split(),verbose=True,quiet=False,logs=False)
#OR
#!pip install plncpro


genome="maize_data/Zea_mays.B73_RefGen_v4.dna.toplevel.1_10.fa"
model="monocot_model/monocot.model"
blastdb="uniprot/uniprotdb"
for i in range(len(gtfList)):
    thisOb=sraObjects[i]
    #first extract transcripts using gffread
    tx_file=thisOb.location+"/transcripts.fa"
    cmd="gffread -w "+tx_file+" -g maize_data/Zea_mays.B73_RefGen_v4.dna.toplevel.1_10.fa "+gtfList[i]
    pe.execute_command(cmd.split(" "),verbose=False,quiet=False,logs=True,objectid=thisOb.srr_accession,command_name="gffread")
    
    #Optional step use biopython to filter transcripts by len
    #out_file=thisOb.location+"/transcripts_filter.fa"
    #output_handle = open(out_file, "w")
    #for record in SeqIO.parse(tx_file, "fasta"):
        # keep tx between 200 and 1000
    #    if len(record)>=500 and len(record)<=1000:
    #        #write to temp file
    #        SeqIO.write(record, output_handle, "fasta")

    
    #run plncpro
    outdir=thisOb.location+"/plncpro_out"
    outfile="plncpro_predictions"
    cmd="plncpro predict -i "+tx_file+" -o "+outdir+" -p "+outfile+" -t 25 --min_len 200 -d "+blastdb+" -m "+model+" -v -r"
    pe.execute_command(cmd.split(),verbose=False,quiet=False,logs=True,objectid=thisOb.srr_accession,command_name="plncpro predict")
        


## Generate reports

In [1]:
!pyrpipe_diagnostic.py report pyrpipe_logs/2020-01-22-18_14_47_pyrpipe.log
!pyrpipe_diagnostic.py benchmark pyrpipe_logs/2020-01-22-18_14_47_pyrpipe.log
!pyrpipe_diagnostic.py shell pyrpipe_logs/2020-01-22-18_14_47_pyrpipe.log


/home/usingh/anaconda3/envs/pyrpipeTest2/lib/python3.5/site-packages/weasyprint/fonts.py:44: UserWarning: @font-face support needs Pango >= 1.38
  warnings.warn('@font-face support needs Pango >= 1.38')
/home/usingh/anaconda3/envs/pyrpipeTest2/lib/python3.5/site-packages/weasyprint/document.py:36: UserWarning: There are known rendering problems and missing features with cairo < 1.15.4. WeasyPrint may work with older versions, but please read the note about the needed cairo version on the "Install" page of the documentation before reporting bugs. http://weasyprint.readthedocs.io/en/latest/install.html
  'There are known rendering problems and missing features with '
Report written to 2020-01-22-18_14_47_pyrpipe.pdf
/home/usingh/anaconda3/envs/pyrpipeTest2/lib/python3.5/site-packages/weasyprint/fonts.py:44: UserWarning: @font-face support needs Pango >= 1.38
  warnings.warn('@font-face support needs Pango >= 1.38')
/home/usingh/anaconda3/envs/pyrpipeTest2/lib/python3.5/site-packages/weas